In [1]:
import org.apache.spark.sql.functions.{broadcast, split, lit}

Intitializing Scala interpreter ...

Spark Web UI available at http://f83b10fae2d6:4040
SparkContext available as 'sc' (version = 3.5.5, master = local[*], app id = local-1754935957994)
SparkSession available as 'spark'


import org.apache.spark.sql.functions.{broadcast, split, lit}


In [2]:
val matchesBucketed = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/matches.csv")
val matchDetailsBucketed =  spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/match_details.csv")

matchesBucketed: org.apache.spark.sql.DataFrame = [match_id: string, mapid: string ... 8 more fields]
matchDetailsBucketed: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 34 more fields]


In [3]:
val medalsmatchesplayersBucketed = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/medals_matches_players.csv")
val medalsBucketed =  spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/medals.csv")

medalsmatchesplayersBucketed: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 2 more fields]
medalsBucketed: org.apache.spark.sql.DataFrame = [medal_id: bigint, sprite_uri: string ... 10 more fields]


In [4]:
import org.apache.spark.sql.functions._
import spark.implicits._

val top10MatchIdsDF =
  matchDetailsBucketed
    .groupBy($"match_id")
    .count()
    .orderBy(desc("count"))
    .limit(10)

top10MatchIdsDF.show(false)   // sanity check

+------------------------------------+-----+
|match_id                            |count|
+------------------------------------+-----+
|4740f30e-2e38-4cae-b858-3ea5a588bc84|24   |
|0a2e9cc8-b84c-46bb-a769-b49dd444a3f0|24   |
|6b4ffe69-a4c1-44b6-9f46-48cad1bd9a14|22   |
|d9780cc1-b5fb-45d6-a194-f7806e769519|22   |
|fcf70b21-3fdf-4957-af2e-21a171424bb3|21   |
|1de1abd4-e662-48a6-8c20-1716cfe7408c|21   |
|308a12a5-f1fa-4002-9088-f2dd0a140aab|21   |
|e9f11daf-0be9-478d-aa27-452399d12258|21   |
|5ed62e3e-56b9-4ebf-8f0e-13707a644b52|21   |
|19f53054-cefc-42be-874c-4dbfea6b27b6|21   |
+------------------------------------+-----+



import org.apache.spark.sql.functions._
import spark.implicits._
top10MatchIdsDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [match_id: string, count: bigint]


In [5]:
matchesBucketed.printSchema()        // See columns and data types
matchDetailsBucketed.printSchema()

root
 |-- match_id: string (nullable = true)
 |-- mapid: string (nullable = true)
 |-- is_team_game: boolean (nullable = true)
 |-- playlist_id: string (nullable = true)
 |-- game_variant_id: string (nullable = true)
 |-- is_match_over: boolean (nullable = true)
 |-- completion_date: timestamp (nullable = true)
 |-- match_duration: string (nullable = true)
 |-- game_mode: string (nullable = true)
 |-- map_variant_id: string (nullable = true)

root
 |-- match_id: string (nullable = true)
 |-- player_gamertag: string (nullable = true)
 |-- previous_spartan_rank: integer (nullable = true)
 |-- spartan_rank: integer (nullable = true)
 |-- previous_total_xp: integer (nullable = true)
 |-- total_xp: integer (nullable = true)
 |-- previous_csr_tier: integer (nullable = true)
 |-- previous_csr_designation: integer (nullable = true)
 |-- previous_csr: integer (nullable = true)
 |-- previous_csr_percent_to_next_tier: integer (nullable = true)
 |-- previous_csr_rank: integer (nullable = true)
 |-

In [6]:
medalsmatchesplayersBucketed.printSchema()        // See columns and data types
medalsBucketed.printSchema()

root
 |-- match_id: string (nullable = true)
 |-- player_gamertag: string (nullable = true)
 |-- medal_id: long (nullable = true)
 |-- count: integer (nullable = true)

root
 |-- medal_id: long (nullable = true)
 |-- sprite_uri: string (nullable = true)
 |-- sprite_left: integer (nullable = true)
 |-- sprite_top: integer (nullable = true)
 |-- sprite_sheet_width: integer (nullable = true)
 |-- sprite_sheet_height: integer (nullable = true)
 |-- sprite_width: integer (nullable = true)
 |-- sprite_height: integer (nullable = true)
 |-- classification: string (nullable = true)
 |-- description: string (nullable = true)
 |-- name: string (nullable = true)
 |-- difficulty: integer (nullable = true)



In [ ]:
# Disabled automatic broadcast join with `spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")`

In [7]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [8]:
val mapsBucketed =  spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/maps.csv")

mapsBucketed: org.apache.spark.sql.DataFrame = [mapid: string, name: string ... 1 more field]


In [9]:
mapsBucketed.printSchema()

root
 |-- mapid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)



In [9]:
# These tables are small (few rows, used for lookup).
# Broadcasting avoids shuffling big tables and makes joins efficient—but only for these, since we disabled auto-broadcast globally.

<console>: 2: error: ';' expected but '#' found.

In [10]:
val medalsBroadcasted = broadcast(medalsBucketed)
val mapsBroadcasted = broadcast(mapsBucketed)

medalsBroadcasted: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [medal_id: bigint, sprite_uri: string ... 10 more fields]
mapsBroadcasted: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [mapid: string, name: string ... 1 more field]


In [11]:
val medalDetails = medalsmatchesplayersBucketed
  .join(medalsBroadcasted, Seq("medal_id"), "left")

medalDetails: org.apache.spark.sql.DataFrame = [medal_id: bigint, match_id: string ... 13 more fields]


In [ ]:
# Explicitly broadcast JOINs `medals` and `maps`

In [12]:
val matchesWithMapName = matchesBucketed
  .join(mapsBroadcasted, Seq("mapid"), "left")

matchesWithMapName: org.apache.spark.sql.DataFrame = [mapid: string, match_id: string ... 10 more fields]


In [ ]:
# Bucket join match_details, matches, and medals_matches_players on match_id with 16 buckets
# Bucketing splits your data into a fixed number of files ("buckets") by the column you choose—here, match_id.
# When you write & then read bucketed tables on the same column, Spark can join them efficiently (avoiding shuffles).

In [ ]:
#  Steps to Bucket the Three Tables
# Write each table as a bucketed table on match_id (16 buckets):

In [16]:
spark.sql("SHOW TABLES IN bootcamp;").show()

+---------+---------------+-----------+
|namespace|      tableName|isTemporary|
+---------+---------------+-----------+
| bootcamp|         events|      false|
| bootcamp|  events_sorted|      false|
| bootcamp|events_unsorted|      false|
+---------+---------------+-----------+



In [ ]:
# spark.sql("DESCRIBE TABLE bootcamp.matches_bucketed")


In [15]:
spark.sql("DROP TABLE IF EXISTS bootcamp.matches_bucketed")
spark.sql("DROP TABLE IF EXISTS bootcamp.match_details_bucketed")
spark.sql("DROP TABLE IF EXISTS bootcamp.medals_matches_players_bucketed")

res7: org.apache.spark.sql.DataFrame = []


In [ ]:
# spark.sql("""DROP TABLE IF EXISTS bootcamp.matches_bucketed""")
# val bucketedDDL = """
# CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
#      match_id STRING,
#      is_team_game BOOLEAN,
#      playlist_id STRING,
#      completion_date TIMESTAMP
#  )
#  USING iceberg
#  PARTITIONED BY (completion_date, bucket(16, match_id));
#  """
#  spark.sql(bucketedDDL)

In [17]:
val matchesDDL = """
CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
     match_id STRING,
     mapid STRING,
     is_team_game BOOLEAN,
     playlist_id STRING,
     game_variant_id STRING,
     is_match_over BOOLEAN,
     completion_date TIMESTAMP
 )
 USING iceberg
 PARTITIONED BY (completion_date, bucket(16, match_id))
"""
spark.sql(matchesDDL)

matchesDDL: String =
"
CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
     match_id STRING,
     mapid STRING,
     is_team_game BOOLEAN,
     playlist_id STRING,
     game_variant_id STRING,
     is_match_over BOOLEAN,
     completion_date TIMESTAMP
 )
 USING iceberg
 PARTITIONED BY (completion_date, bucket(16, match_id))
"
res9: org.apache.spark.sql.DataFrame = []


In [18]:
// See the table structure (columns, partitioning)
spark.sql("DESCRIBE FORMATTED bootcamp.matches_bucketed").show(100, false)

+----------------------------+-------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                              |comment|
+----------------------------+-------------------------------------------------------------------------------------------------------+-------+
|match_id                    |string                                                                                                 |NULL   |
|mapid                       |string                                                                                                 |NULL   |
|is_team_game                |boolean                                                                                                |NULL   |
|playlist_id                 |string                                                                                                 |NULL   |

In [19]:
spark.table("bootcamp.matches_bucketed").count()

res11: Long = 0


In [20]:
matchesWithMapName.count()

res12: Long = 24025


In [21]:
val matchDetailsDDL = """
CREATE TABLE IF NOT EXISTS bootcamp.match_details_bucketed (
     match_id STRING,
     player_gamertag STRING,
     player_total_kills INT,
     player_total_deaths INT,
     player_total_assists INT,
     team_id INT
     -- (add more columns if needed, matching your data)
 )
 USING iceberg
 PARTITIONED BY (bucket(16, match_id))
"""
spark.sql(matchDetailsDDL)

matchDetailsDDL: String =
"
CREATE TABLE IF NOT EXISTS bootcamp.match_details_bucketed (
     match_id STRING,
     player_gamertag STRING,
     player_total_kills INT,
     player_total_deaths INT,
     player_total_assists INT,
     team_id INT
     -- (add more columns if needed, matching your data)
 )
 USING iceberg
 PARTITIONED BY (bucket(16, match_id))
"
res13: org.apache.spark.sql.DataFrame = []


In [22]:
val medalsMatchesPlayersDDL = """
CREATE TABLE IF NOT EXISTS bootcamp.medals_matches_players_bucketed (
     match_id STRING,
     player_gamertag STRING,
     medal_id LONG,
     count INT
 )
 USING iceberg
 PARTITIONED BY (bucket(16, match_id))
"""
spark.sql(medalsMatchesPlayersDDL)

medalsMatchesPlayersDDL: String =
"
CREATE TABLE IF NOT EXISTS bootcamp.medals_matches_players_bucketed (
     match_id STRING,
     player_gamertag STRING,
     medal_id LONG,
     count INT
 )
 USING iceberg
 PARTITIONED BY (bucket(16, match_id))
"
res14: org.apache.spark.sql.DataFrame = []


In [23]:
matchesBucketed.count()

res15: Long = 24025


In [ ]:
# val batchSize = 1000 // Adjust based on your cluster resources

# matchesBucketed
#   .repartition(10) // Adjust partition count based on your cluster
#   .select(
#     $"match_id",
#     $"mapid", 
#     $"is_team_game",
#     $"playlist_id",
#     $"game_variant_id",
#     $"is_match_over",
#     $"completion_date"
#   )
#   .write
#   .mode("append")
#   .option("write.distribution-mode", "hash") // Use hash distribution for Iceberg
#   .insertInto("bootcamp.matches_bucketed")

In [ ]:
# matchesBucketed
#   .select(
#     $"match_id",
#     $"mapid",
#     $"is_team_game", 
#     $"playlist_id",
#     $"game_variant_id",
#     $"is_match_over",
#     $"completion_date"
#   )
#   .write
#   .mode("append")
#   .option("write.distribution-mode", "hash")
#   .option("write.fanout.enabled", "true") // Enable fanout for better performance
#   .option("write.target-file-size-bytes", "134217728") // 128MB target file size
#   .insertInto("bootcamp.matches_bucketed")

In [37]:
spark.table("bootcamp.matches_bucketed").count()

res27: Long = 0


In [22]:
# import spark.implicits._

# // Define the IDs you care about
# val matchIds = Seq(
#   "4740f30e-2e38-4cae-b858-3ea5a588bc84",
#   "0a2e9cc8-b84c-46bb-a769-b49dd444a3f0",
#   "6b4ffe69-a4c1-44b6-9f46-48cad1bd9a14",
#   "d9780cc1-b5fb-45d6-a194-f7806e769519",
#   "fcf70b21-3fdf-4957-af2e-21a171424bb3"
# )

# // Filter and count
# val filteredCount = matchesBucketed
#   .filter($"match_id".isin(matchIds: _*))
#   .count()

# println(s"Total rows for given match_ids: $filteredCount")

Total rows for given match_ids: 5


import spark.implicits._
matchIds: Seq[String] = List(4740f30e-2e38-4cae-b858-3ea5a588bc84, 0a2e9cc8-b84c-46bb-a769-b49dd444a3f0, 6b4ffe69-a4c1-44b6-9f46-48cad1bd9a14, d9780cc1-b5fb-45d6-a194-f7806e769519, fcf70b21-3fdf-4957-af2e-21a171424bb3)
filteredCount: Long = 5


In [24]:
import org.apache.spark.sql.functions._
import spark.implicits._

// 1) Count rows per match_id and take top 100
val top100MatchCounts =
  matchDetailsBucketed
    .groupBy($"match_id")
    .count()
    .orderBy(desc("count"))
    .limit(100)

// See the 100 IDs and their counts
top100MatchCounts.show(false)

// Keep just the IDs for filtering/joins
val top100IdsDF = top100MatchCounts.select($"match_id")

+------------------------------------+-----+
|match_id                            |count|
+------------------------------------+-----+
|4740f30e-2e38-4cae-b858-3ea5a588bc84|24   |
|0a2e9cc8-b84c-46bb-a769-b49dd444a3f0|24   |
|6b4ffe69-a4c1-44b6-9f46-48cad1bd9a14|22   |
|d9780cc1-b5fb-45d6-a194-f7806e769519|22   |
|fcf70b21-3fdf-4957-af2e-21a171424bb3|21   |
|1de1abd4-e662-48a6-8c20-1716cfe7408c|21   |
|308a12a5-f1fa-4002-9088-f2dd0a140aab|21   |
|e9f11daf-0be9-478d-aa27-452399d12258|21   |
|5ed62e3e-56b9-4ebf-8f0e-13707a644b52|21   |
|19f53054-cefc-42be-874c-4dbfea6b27b6|21   |
|41c411e0-7273-4954-bbf3-1720667f0f75|20   |
|2895a8b8-ffbd-49e8-8863-ffe0f319636d|20   |
|614f85c6-2846-4d2a-bbd6-21890465655f|20   |
|afee7207-db02-437d-b805-df39374ad7be|20   |
|aee287dc-d260-4c9b-ba11-c5ce0592ec13|20   |
|a9317e5b-4aa5-40b0-a736-5e5417a2c980|20   |
|d37847b8-92f2-48b6-8848-0ed2e9ee00cd|20   |
|123c93eb-522f-408c-9b2d-8c1012be9b3b|20   |
|59e9dc52-5065-402d-a41a-81bbc15a085a|20   |
|a6af8f27-

import org.apache.spark.sql.functions._
import spark.implicits._
top100MatchCounts: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [match_id: string, count: bigint]
top100IdsDF: org.apache.spark.sql.DataFrame = [match_id: string]


In [25]:
// Filter the matches CSV to only the top-100 match_ids
val matchesTop100 =
  matchesBucketed
    .join(top100IdsDF, Seq("match_id"), "inner")
    .select(
      $"match_id",
      $"mapid",
      $"is_team_game",
      $"playlist_id",
      $"game_variant_id",
      $"is_match_over",
      $"completion_date"
    )

// Optional sanity checks
println(s"Rows to load: ${matchesTop100.count()}")
matchesTop100.show(5, truncate = false)

// Insert ONLY these rows into the Iceberg table
matchesTop100
  .write
  .mode("append")
  .insertInto("bootcamp.matches_bucketed")

Rows to load: 100
+------------------------------------+------------------------------------+------------+------------------------------------+------------------------------------+-------------+-------------------+
|match_id                            |mapid                               |is_team_game|playlist_id                         |game_variant_id                     |is_match_over|completion_date    |
+------------------------------------+------------------------------------+------------+------------------------------------+------------------------------------+-------------+-------------------+
|0001a1c4-83dc-4f40-a97e-7910a765c96a|c7805740-f206-11e4-982c-24be05e24f7e|true        |780cc101-005c-4fca-8ce7-6f36d7156ffe|b45854a7-e6e1-4a9c-9104-139934511779|true         |2016-01-06 00:00:00|
|035f360e-989b-4cea-bb71-9c70a236d0f0|c7805740-f206-11e4-982c-24be05e24f7e|true        |780cc101-005c-4fca-8ce7-6f36d7156ffe|8d4a3dbc-ef7a-405e-862b-34093ff582fd|true         |2016-02-20 00:00:0

matchesTop100: org.apache.spark.sql.DataFrame = [match_id: string, mapid: string ... 5 more fields]


In [20]:
# matchesBucketed
#   .limit(100)
#   .select(
#     $"match_id",
#     $"mapid",
#     $"is_team_game",
#     $"playlist_id",
#     $"game_variant_id",
#     $"is_match_over",
#     $"completion_date"
#   )
#   .write
#   .mode("append")
#   .insertInto("bootcamp.matches_bucketed")

In [40]:
spark.table("bootcamp.matches_bucketed").show(5)

+--------------------+--------------------+------------+--------------------+--------------------+-------------+-------------------+
|            match_id|               mapid|is_team_game|         playlist_id|     game_variant_id|is_match_over|    completion_date|
+--------------------+--------------------+------------+--------------------+--------------------+-------------+-------------------+
|5c4309f2-9150-4e1...|c74c9d0f-f206-11e...|        true|780cc101-005c-4fc...|b45854a7-e6e1-4a9...|         true|2016-01-17 00:00:00|
|d33e5f70-f75a-4c4...|c74c9d0f-f206-11e...|        true|0bcf2be1-3168-4e4...|b45854a7-e6e1-4a9...|         true|2015-11-20 00:00:00|
|b60ac716-e437-427...|c74c9d0f-f206-11e...|        true|780cc101-005c-4fc...|b45854a7-e6e1-4a9...|         true|2016-02-19 00:00:00|
|5dbe750f-95c8-45d...|c7b7baf0-f206-11e...|        true|780cc101-005c-4fc...|b45854a7-e6e1-4a9...|         true|2016-02-19 00:00:00|
|f9476ecb-2144-49a...|c74c9d0f-f206-11e...|        NULL|0bcf2be1-3168

In [23]:
# spark.sql("DROP TABLE IF EXISTS bootcamp.matches_bucketed")

res15: org.apache.spark.sql.DataFrame = []


In [ ]:
# matchesBucketed
#   .select($"match_id", $"mapid", $"is_team_game", $"playlist_id", $"game_variant_id", $"is_match_over", $"completion_date")
#   .write
#   .mode("append")
#   .partitionBy("completion_date")
#   .bucketBy(16, "match_id").saveAsTable("bootcamp.matches_bucketed")

In [26]:
spark.sql("DESCRIBE TABLE EXTENDED bootcamp.matches_bucketed")

res17: org.apache.spark.sql.DataFrame = [col_name: string, data_type: string ... 1 more field]


In [27]:
spark.table("bootcamp.matches_bucketed").count()

res18: Long = 100


In [28]:
spark.table("bootcamp.matches_bucketed").show(5)

+--------------------+--------------------+------------+--------------------+--------------------+-------------+-------------------+
|            match_id|               mapid|is_team_game|         playlist_id|     game_variant_id|is_match_over|    completion_date|
+--------------------+--------------------+------------+--------------------+--------------------+-------------+-------------------+
|5c4309f2-9150-4e1...|c74c9d0f-f206-11e...|        true|780cc101-005c-4fc...|b45854a7-e6e1-4a9...|         true|2016-01-17 00:00:00|
|d33e5f70-f75a-4c4...|c74c9d0f-f206-11e...|        true|0bcf2be1-3168-4e4...|b45854a7-e6e1-4a9...|         true|2015-11-20 00:00:00|
|b60ac716-e437-427...|c74c9d0f-f206-11e...|        true|780cc101-005c-4fc...|b45854a7-e6e1-4a9...|         true|2016-02-19 00:00:00|
|5dbe750f-95c8-45d...|c7b7baf0-f206-11e...|        true|780cc101-005c-4fc...|b45854a7-e6e1-4a9...|         true|2016-02-19 00:00:00|
|f9476ecb-2144-49a...|c74c9d0f-f206-11e...|        NULL|0bcf2be1-3168

In [29]:
matchDetailsBucketed.count()

res20: Long = 151761


In [25]:
# matchDetailsBucketed
#   .limit(114)
#   .select($"match_id", $"player_gamertag", $"player_total_kills", $"player_total_deaths", $"player_total_assists", $"team_id" /*, ...other cols*/)
#   .write
#   .mode("append")
#   .insertInto("bootcamp.match_details_bucketed")

In [30]:
val selectedMatchIds = Seq(
  "4740f30e-2e38-4cae-b858-3ea5a588bc84",
  "0a2e9cc8-b84c-46bb-a769-b49dd444a3f0",
  "6b4ffe69-a4c1-44b6-9f46-48cad1bd9a14",
  "d9780cc1-b5fb-45d6-a194-f7806e769519",
  "fcf70b21-3fdf-4957-af2e-21a171424bb3",
  "1de1abd4-e662-48a6-8c20-1716cfe7408c",
  "308a12a5-f1fa-4002-9088-f2dd0a140aab",
  "e9f11daf-0be9-478d-aa27-452399d12258",
  "5ed62e3e-56b9-4ebf-8f0e-13707a644b52",
  "19f53054-cefc-42be-874c-4dbfea6b27b6",
  "41c411e0-7273-4954-bbf3-1720667f0f75",
  "2895a8b8-ffbd-49e8-8863-ffe0f319636d",
  "614f85c6-2846-4d2a-bbd6-21890465655f",
  "afee7207-db02-437d-b805-df39374ad7be",
  "aee287dc-d260-4c9b-ba11-c5ce0592ec13",
  "a9317e5b-4aa5-40b0-a736-5e5417a2c980",
  "d37847b8-92f2-48b6-8848-0ed2e9ee00cd",
  "123c93eb-522f-408c-9b2d-8c1012be9b3b",
  "59e9dc52-5065-402d-a41a-81bbc15a085a",
  "a6af8f27-2f99-4746-aebf-cb86790b3d74"
)


selectedMatchIds: Seq[String] = List(4740f30e-2e38-4cae-b858-3ea5a588bc84, 0a2e9cc8-b84c-46bb-a769-b49dd444a3f0, 6b4ffe69-a4c1-44b6-9f46-48cad1bd9a14, d9780cc1-b5fb-45d6-a194-f7806e769519, fcf70b21-3fdf-4957-af2e-21a171424bb3, 1de1abd4-e662-48a6-8c20-1716cfe7408c, 308a12a5-f1fa-4002-9088-f2dd0a140aab, e9f11daf-0be9-478d-aa27-452399d12258, 5ed62e3e-56b9-4ebf-8f0e-13707a644b52, 19f53054-cefc-42be-874c-4dbfea6b27b6, 41c411e0-7273-4954-bbf3-1720667f0f75, 2895a8b8-ffbd-49e8-8863-ffe0f319636d, 614f85c6-2846-4d2a-bbd6-21890465655f, afee7207-db02-437d-b805-df39374ad7be, aee287dc-d260-4c9b-ba11-c5ce0592ec13, a9317e5b-4aa5-40b0-a736-5e5417a2c980, d37847b8-92f2-48b6-8848-0ed2e9ee00cd, 123c93eb-522f-408c-9b2d-8c1012be9b3b, 59e9dc52-5065-402d-a41a-81bbc15a085a, a6af8f27-2f99-4746-aebf-cb86790b3d74)


In [31]:
val filteredMatchDetails = matchDetailsBucketed
  .filter($"match_id".isin(selectedMatchIds: _*))
  .select(
    $"match_id",
    $"player_gamertag",
    $"player_total_kills",
    $"player_total_deaths",
    $"player_total_assists",
    $"team_id"
  )


filteredMatchDetails: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 4 more fields]


In [32]:
println(s"Rows to load: ${filteredMatchDetails.count()}")
println(s"Distinct match_ids in load: ${filteredMatchDetails.select('match_id).distinct().count()}")


Rows to load: 418
Distinct match_ids in load: 20


In [33]:
// Step 3 — Insert into Iceberg table
filteredMatchDetails
  .write
  .mode("append")
  .insertInto("bootcamp.match_details_bucketed")

In [34]:
spark.table("bootcamp.match_details_bucketed").count()

res23: Long = 418


In [35]:
spark.table("bootcamp.match_details_bucketed").show(5)

+--------------------+---------------+------------------+-------------------+--------------------+-------+
|            match_id|player_gamertag|player_total_kills|player_total_deaths|player_total_assists|team_id|
+--------------------+---------------+------------------+-------------------+--------------------+-------+
|a6af8f27-2f99-474...|      Sevenface|                31|                 10|                  15|      1|
|a6af8f27-2f99-474...|  Little Osorio|                 1|                  6|                   0|      0|
|a6af8f27-2f99-474...|    Wonderboy92|                26|                 20|                   5|      0|
|a6af8f27-2f99-474...|  Dogie Dog 101|                32|                 13|                   4|      1|
|a6af8f27-2f99-474...|    Mario Chief|                 1|                  1|                   0|      1|
+--------------------+---------------+------------------+-------------------+--------------------+-------+
only showing top 5 rows



In [51]:
# spark.sql("SHOW PARTITIONS bootcamp.match_details_bucketed").show(100, false)

<console>: 2: error: ';' expected but '#' found.

In [36]:
medalsmatchesplayersBucketed.count()

res25: Long = 755229


In [29]:
# medalsmatchesplayersBucketed
#    .limit(126)
#   .select($"match_id", $"player_gamertag", $"medal_id", $"count")
#   .write
#   .mode("append")
#   .insertInto("bootcamp.medals_matches_players_bucketed")

In [37]:
// Step 2 — filter medalsmatchesplayersBucketed to only those IDs
val filteredMedalsMatchesPlayers = medalsmatchesplayersBucketed
  .filter($"match_id".isin(selectedMatchIds: _*))
  .select(
    $"match_id",
    $"player_gamertag",
    $"medal_id",
    $"count"
  )

println(s"Rows to load: ${filteredMedalsMatchesPlayers.count()}")
println(s"Distinct match_ids in load: ${filteredMedalsMatchesPlayers.select('match_id).distinct().count()}")

// Step 3 — write to Iceberg table
filteredMedalsMatchesPlayers
  .write
  .mode("append")
  .insertInto("bootcamp.medals_matches_players_bucketed")

Rows to load: 2801
Distinct match_ids in load: 20


filteredMedalsMatchesPlayers: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 2 more fields]


In [38]:
spark.table("bootcamp.medals_matches_players_bucketed").count()

res27: Long = 2801


In [39]:
spark.table("bootcamp.medals_matches_players_bucketed").show(5)

+--------------------+---------------+----------+-----+
|            match_id|player_gamertag|  medal_id|count|
+--------------------+---------------+----------+-----+
|a9317e5b-4aa5-40b...|    ILLICIT 117|3261908037|    6|
|a9317e5b-4aa5-40b...|    ILLICIT 117|1080468863|    1|
|a9317e5b-4aa5-40b...|    ILLICIT 117|3400287617|    1|
|a9317e5b-4aa5-40b...|    ILLICIT 117|1219497744|    2|
|a9317e5b-4aa5-40b...|    ILLICIT 117|2287626681|    1|
+--------------------+---------------+----------+-----+
only showing top 5 rows



In [40]:
spark.sql("SHOW PARTITIONS bootcamp.medals_matches_players_bucketed").show(100, false)

org.apache.spark.sql.catalyst.ExtendedAnalysisException:  [INVALID_PARTITION_OPERATION.PARTITION_MANAGEMENT_IS_UNSUPPORTED] The partition command is invalid. Table `demo`.`bootcamp`.`medals_matches_players_bucketed` does not support partition management.; line 1 pos 16;

In [ ]:
# Analytical Aggregations with Joins
# First, read the bucketed tables into DataFrames:

In [41]:
val matchesDF = spark.table("bootcamp.matches_bucketed")
val matchDetailsDF = spark.table("bootcamp.match_details_bucketed")
val medalsMatchesPlayersDF = spark.table("bootcamp.medals_matches_players_bucketed")

matchesDF: org.apache.spark.sql.DataFrame = [match_id: string, mapid: string ... 5 more fields]
matchDetailsDF: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 4 more fields]
medalsMatchesPlayersDF: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 2 more fields]


In [42]:
// Join all three tables on match_id (bucket join)
val joinedDF = matchDetailsDF
  .join(matchesDF, Seq("match_id"), "inner")
  .join(medalsMatchesPlayersDF, Seq("match_id", "player_gamertag"), "left")

joinedDF: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 12 more fields]


In [43]:
joinedDF.count()

res30: Long = 2856


In [44]:
joinedDF.printSchema()

root
 |-- match_id: string (nullable = true)
 |-- player_gamertag: string (nullable = true)
 |-- player_total_kills: integer (nullable = true)
 |-- player_total_deaths: integer (nullable = true)
 |-- player_total_assists: integer (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- mapid: string (nullable = true)
 |-- is_team_game: boolean (nullable = true)
 |-- playlist_id: string (nullable = true)
 |-- game_variant_id: string (nullable = true)
 |-- is_match_over: boolean (nullable = true)
 |-- completion_date: timestamp (nullable = true)
 |-- medal_id: long (nullable = true)
 |-- count: integer (nullable = true)



In [45]:
// Which player averages the most kills per game?
val playerAvgKills =
  joinedDF
    .groupBy($"player_gamertag")
    .agg(
      avg($"player_total_kills").as("avg_kills_per_game"),
      countDistinct($"match_id").as("games_played")
    )
    .orderBy(desc("avg_kills_per_game"))

playerAvgKills.show(20, truncate = false)

+---------------+------------------+------------+
|player_gamertag|avg_kills_per_game|games_played|
+---------------+------------------+------------+
|Babtou SeGpa   |42.0              |1           |
|Deceit         |39.0              |1           |
|NILLOC 916     |35.0              |1           |
|EquallyStupid  |34.0              |1           |
|Hatter Matigan |34.0              |1           |
|BooneYO        |34.0              |1           |
|Starkey Xx     |33.0              |1           |
|hreskofreight4 |33.0              |1           |
|Hydrix         |32.0              |1           |
|Eh Snipe       |32.0              |1           |
|Dogie Dog 101  |32.0              |1           |
|Statvr         |32.0              |1           |
|Own U          |32.0              |1           |
|Sevenface      |31.0              |1           |
|sitbird        |30.0              |1           |
|Boricpapergod  |28.0              |1           |
|TRANZ4M3R      |28.0              |1           |


playerAvgKills: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [player_gamertag: string, avg_kills_per_game: double ... 1 more field]


In [46]:
//Q2) Which playlist gets played the most?
val mostPlayedPlaylist =
  joinedDF
    .select($"playlist_id", $"match_id")
    .distinct()                         // avoid counting players as extra matches
    .groupBy($"playlist_id")
    .agg(count($"match_id").as("num_matches"))
    .orderBy(desc("num_matches"))

mostPlayedPlaylist.show(20, truncate = false)

+------------------------------------+-----------+
|playlist_id                         |num_matches|
+------------------------------------+-----------+
|780cc101-005c-4fca-8ce7-6f36d7156ffe|20         |
+------------------------------------+-----------+



mostPlayedPlaylist: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [playlist_id: string, num_matches: bigint]


In [47]:
//Q3) Which map gets played the most?
val mostPlayedMap =
  joinedDF
    .select($"mapid", $"match_id")
    .distinct()
    .groupBy($"mapid")
    .agg(count($"match_id").as("num_matches"))
    .orderBy(desc("num_matches"))

mostPlayedMap.show(20, truncate = false)

+------------------------------------+-----------+
|mapid                               |num_matches|
+------------------------------------+-----------+
|c7805740-f206-11e4-982c-24be05e24f7e|11         |
|c74c9d0f-f206-11e4-8330-24be05e24f7e|7          |
|c7b7baf0-f206-11e4-ae9a-24be05e24f7e|2          |
+------------------------------------+-----------+



mostPlayedMap: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [mapid: string, num_matches: bigint]


In [48]:
//Q4) Which map do players get the most Killing Spree medals on?
//here we lookup the medal_id for “Killing Spree” from your medals table, then filter and aggregate.
// 1) find the medal_id for "Killing Spree"
val killingSpreeId: Long =
  medalsBucketed.filter(lower($"name") === "killing spree")
    .select($"medal_id")
    .as[Long]
    .head()

// 2) filter joinedDF to that medal and aggregate per map
val killingSpreeByMap =
  joinedDF
    .filter($"medal_id" === killingSpreeId)
    .groupBy($"mapid")
    .agg(sum($"count").as("total_killing_sprees"))
    .orderBy(desc("total_killing_sprees"))

killingSpreeByMap.show(20, truncate = false)

// (optional) add map name
val killingSpreeByMapNamed =
  killingSpreeByMap
    .join(broadcast(mapsBucketed.select($"mapid", $"name")), Seq("mapid"), "left")
    .select($"mapid", $"name", $"total_killing_sprees")
    .orderBy(desc("total_killing_sprees"))

killingSpreeByMapNamed.show(20, truncate = false)

+------------------------------------+--------------------+
|mapid                               |total_killing_sprees|
+------------------------------------+--------------------+
|c7805740-f206-11e4-982c-24be05e24f7e|87                  |
|c74c9d0f-f206-11e4-8330-24be05e24f7e|58                  |
|c7b7baf0-f206-11e4-ae9a-24be05e24f7e|21                  |
+------------------------------------+--------------------+

+------------------------------------+--------+--------------------+
|mapid                               |name    |total_killing_sprees|
+------------------------------------+--------+--------------------+
|c7805740-f206-11e4-982c-24be05e24f7e|Glacier |87                  |
|c74c9d0f-f206-11e4-8330-24be05e24f7e|Alpine  |58                  |
|c7b7baf0-f206-11e4-ae9a-24be05e24f7e|Parallax|21                  |
+------------------------------------+--------+--------------------+



killingSpreeId: Long = 2430242797
killingSpreeByMap: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [mapid: string, total_killing_sprees: bigint]
killingSpreeByMapNamed: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [mapid: string, name: string ... 1 more field]


In [49]:
//Step 5: Try different .sortWithinPartitions and compare file sizes
//We’ll write the same aggregated dataset out with different sorts and compare total bytes on disk. 
//Since playlists/maps are low cardinality, sorting by them can improve compression.
import org.apache.hadoop.fs.{FileSystem, Path}
import org.apache.hadoop.conf.Configuration

def writeAndSize(dfName: String, df: org.apache.spark.sql.DataFrame, path: String): Long = {
  df.write.mode("overwrite").parquet(path)
  val fs = FileSystem.get(new Configuration())
  val statuses = fs.listStatus(new Path(path))
  val total = statuses.filter(s => s.isFile && s.getPath.getName.endsWith(".parquet")).map(_.getLen).sum
  println(f"$dfName wrote ${(total/1024.0/1024.0)}%.2f MB to $path")
  total
}

import org.apache.hadoop.fs.{FileSystem, Path}
import org.apache.hadoop.conf.Configuration
writeAndSize: (dfName: String, df: org.apache.spark.sql.DataFrame, path: String)Long


In [51]:
//Choose an “aggregated dataset” to test (e.g., player averages)
//scala
val baseAgg = playerAvgKills // any aggregated DF you want to compare

baseAgg: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [player_gamertag: string, avg_kills_per_game: double ... 1 more field]


In [52]:
//Compare no sort vs sorts

val outBase = "/tmp/agg_nosort"
val outByPlaylist = "/tmp/agg_sort_playlist"
val outByMap = "/tmp/agg_sort_map"
val outByBoth = "/tmp/agg_sort_both"

val szNoSort   = writeAndSize("no sort", baseAgg, outBase)
val szPlaylist = writeAndSize("sortWithinPartitions(playlist_id)", 
  baseAgg.sortWithinPartitions($"player_gamertag"), // if baseAgg doesn’t have playlist_id/mapid, pick the low-card columns in *that* DF
  outByPlaylist
)

val szMap = writeAndSize("sortWithinPartitions(mapid)",
  baseAgg.sortWithinPartitions($"player_gamertag"), // adjust column choice to something low-cardinality present in baseAgg
  outByMap
)

// Example if your aggregated DF includes both columns; if not, try this on `mostPlayedMapNamed` or build a DF that has both
// val szBoth = writeAndSize("sortWithinPartitions(playlist_id,mapid)",
//   someDF.sortWithinPartitions($"playlist_id", $"mapid"),
//   outByBoth
// )

no sort wrote 0.01 MB to /tmp/agg_nosort
sortWithinPartitions(playlist_id) wrote 0.01 MB to /tmp/agg_sort_playlist
sortWithinPartitions(mapid) wrote 0.01 MB to /tmp/agg_sort_map


outBase: String = /tmp/agg_nosort
outByPlaylist: String = /tmp/agg_sort_playlist
outByMap: String = /tmp/agg_sort_map
outByBoth: String = /tmp/agg_sort_both
szNoSort: Long = 6995
szPlaylist: Long = 7084
szMap: Long = 7084
